# Exercice 3

---

Le but de ce notebook est d'étudier le jeu de données de la valeur des crypto afin de pouvoir faire une prédiction des gains possibles qu'on pourrait avoir.

Pour des raisons de taille, le jeu de données n'a pas pu être uploadé avec le github. Il faudra donc le récupérer à [l'adresse suivante](https://www.kaggle.com/c/g-research-crypto-forecasting/data). Seul les jeux de données train.csv et asset_details.csv nous intéressent.

# Import des bibliothèques

In [ ]:
from datetime import datetime
import dateutil.tz
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

# Chargement des tables dans l'espace de travail

In [ ]:
crypto_train = pd.read_csv("g-research-crypto-forecasting/train.csv")
# supplemental_train est une donnée d'entrainement fournie pour indication. Ne pas utiliser.
#crypto_additional_train = pd.read_csv("g-research-crypto-forecasting/supplemental_train.csv")
crypto_info = pd.read_csv("g-research-crypto-forecasting/asset_details.csv")

In [ ]:
# Piège : la colonne timestamp possède une valeur assez peu lisible.
crypto_train.head(5)

# Partie 1 - Qualité des données

1) Trouvez le nombre de lignes et de colonnes.
1) Faites un join avec la table des infos pour restituer à chaque cryptomonnaie son nom.
1) Trouvez le nombre de valeur manquantes. Quelle est la crypto-monnaie avec le plus de valeurs manquantes ?
1) Créer une colonne date qui change la colonne timestamp en une vraie date.
1) Timestamp maximal, minimal et nombre de points pour chacun de nos assets. Quelle semble être la granularité de notre timeseries ?
1) Pour chacune des crypto-monnaies, de combien de points disposons-nous ? Que pouvons-nous dire en comparant avec la date minimal pour chacune de nos cryptomonnaies ?

# Partie 2 - Analyse données

---

## Explication des valeurs des colonnes.

- **timestamp** : La minute considérée par la colonne.
- **Asset_ID** & **Asset_Name** : identifiant de la crypto-monnaie.
- **Count** : Nombre d'échanges de la monnaie lors de la minute.
- **Open** & **Close** : Prix d'ouverture et de fermeture (prix au début et à la fin de la minute).
- **High** & **Low** : Prix maximal et minimal atteint dans la minute.
- **Volume** : Montant total échangé lors de la minute.
- **VWAP** : Prix moyen sur la minute pondéré par volume de vente.
- **Target** : Log du retour sur investissement sur les 15 prochaines minutes auquel on a appliqué une transformation. f(log(prix à l'instant t+16) - log(prix à l'instant t+1))
- **Weight** : Lié à la cryptomonnaie. Importance de la crypto-monnaie (valeur arbitraire).

---

Nous n'allons pas utiliser les valeurs de weight et target. Nous allons essayer de prédire la VWAP de la monnaie avec 7j d'avance.

---

1) Calculez la corrélations entre les valeurs moyennes des différentes monnaies sur l'année 2021. Ce résultat vous étonne-t-il ?
1) Au vue de la question précédente, nous allons nous intéresser uniquement au bitcoin pour l'instant. Identifiez les timestamp manquantes pour le bitcoin.
1) Quand sont atteints les optimums et pourquoi ? Quand sont atteint les plus grands écarts min max en valeur puis en %
1) Quels jours de la semaine, quels mois de l'année et quelles semaines de l'années y a-t-il le plus de transactions (en moyenne) ? Décomposez les statistiques selon les périodes qui vous semblent pertinentes.
1) Faire un graphe qui présente par semaine le prix moyen des cryptomonnaies avec leur min et le max (dans le même graphe).
1) Autocorrelation décalée pour voir s'il y a des périodicités.
1) Faire des graphe qui présentent la tendance et  la saisonnalité des données.
1) Quelle méthode utiliseriez-vous pour faire jeux de test et train ?

# Partie 3 - Prédiction

---

Notre but est de prédire la VWAP avec une semaine d'avance.
Pour se simplifier la tâche, nous allons agglomérer les données à la maille jour avec le max qui est le max de la journée, le min est le min, le volume est la somme des volumes, la VWAP qui est la VWAP pondérée...

---

1) Mettez en forme le jeu de données et créez la variable cible à prédire.
1) Rajoutez la semaine de l'année en tant que variable catégorique.
1) Afin de préparer les prédictions, normalisez les données.
1) Ajoutez comme features un historique des valeurs sur les dernières jours et toutes autres valeurs qui semblent intéressantes. (Dans les problèmes à saisonnalité, on aurait récupéré les valeurs des précédentes saisons)
1) En choisissant judicieusement vos jeux d'entrainement et de tests, faites des prédictions en utilisant une régression linéaire. Evaluez les résultats.
1) Transformez la variable cible, au lieu de prédire la valeur à jour + 1, nous allons prédire la différence entre valeur actuelle et valeur à jour + 1. Comparez les résultats, conclure.
- SARIMAX (ne fonctionne que pour les données stationnaires + ne prend pas en compte les différentes variables) ==> Inutile ici. Notre problème n'est pas stationnaire.
- Modèles "classiques". Problème : on ne prédit pas très bien le rapport entre les features. 